In [1]:
import glob
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.tree import export_graphviz, DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN
from sklearn import tree
from scipy import stats
import ipaddress
import pickle
import re
import sys
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight

pd.options.mode.chained_assignment = None
from IPython.display import display, HTML

In [2]:
b_classes = ['normal', 'malware']  ### 0: Normal   1: Malware
m_classes = ['normal', 'backdoor', 'dos', 'ddos', 'injection', 'mitm', 'password', 'ransomware', 'scanning', 'xss']
g_classes = ['normal', 'backdoor', 'dos', 'others'] 

In [3]:
b_classes_df = pd.DataFrame(b_classes, columns=['class'])
m_classes_df = pd.DataFrame(m_classes, columns=['class'])
g_classes_df = pd.DataFrame(g_classes, columns=['class'])

# Functions

In [5]:
def save_model(RF, filename):

    pickle.dump(RF, open(filename, 'wb'))

In [4]:
"""
Function to prepare the classes to be classified.
"""
def prepare_new_classes(data):
    data['g_class'] = [-1]*len(data)
    data['g_class'] = np.where(((data['type']=='normal') | (data['type']=='backdoor') | (data['type']=='dos') ), data['type'], 'others')
    return data

In [6]:
"""
Function to Fit model based on optimal values of depth and number of estimators and use it
to compute feature importance for all the features.
"""
def get_feature_importance(depth, n_tree, max_leaf, X_train, y_train):
    
    rf_opt = RandomForestClassifier(max_depth = depth, n_estimators = n_tree, max_leaf_nodes=max_leaf, random_state=42, bootstrap=False)
    rf_opt.fit(X_train, y_train)
    feature_importance = pd.DataFrame(rf_opt.feature_importances_)
    feature_importance.index = X_train.columns
    feature_importance = feature_importance.sort_values(by=list(feature_importance.columns),axis=0,ascending=False)
    
    return feature_importance

In [7]:
"""
Function to Fit model based on optimal values of depth and number of estimators and feature importance
to find the fewest possible features to exceed the previously attained score with all selected features
"""
def get_fewest_features(depth, n_tree, max_leaf, importance):    
    sorted_feature_names = importance.index
    print('sorted_feature_names: ', sorted_feature_names)
    features = []
    for f in range(1,len(sorted_feature_names)+1):
        features.append(sorted_feature_names[0:f])
    print('features:', features)
    return features

In [9]:
"""
Function to calculate the score of the RF model with the given
depth, number of trees, and features
"""
def get_scores(classes, depth, n_tree, feats, max_leaf, X_train, y_train, X_test, y_test):
    model = RandomForestClassifier(max_depth=depth, n_estimators = n_tree, max_leaf_nodes=max_leaf, n_jobs=4,
                                    random_state=42, bootstrap=False)
    # scores = cross_val_score(rf_try, np.array(features[sorted_feature_names[0:f]]), 
    #                         Label, cv=5, scoring='f1_macro')
    
    model.fit(X_train[feats], y_train)
    y_pred = model.predict(X_test[feats])

    class_report = classification_report(y_test, y_pred, target_names=classes, output_dict = True)
    #print(classification_report)
    macro_score = class_report['macro avg']['f1-score']
    weighted_score = class_report['weighted avg']['f1-score']

    return model, class_report, macro_score, weighted_score, y_pred

In [10]:
"""
Function to get train data features and labels
"""
def get_xtrain_ytrain(Train, classification_type, feat_num):
    
    if feat_num == 16:
        X_train = Train[['ip.len', 'tcp.flags.syn', 'tcp.flags.ack', 'tcp.flags.push',
                    'tcp.flags.fin', 'tcp.flags.reset', 'tcp.flags.ece', 'ip.proto', 'ip.ttl', 'tcp.window_size_value',
                    'tcp.hdr_len', 'udp.length','srcport', 'dstport']]
    else:
        X_train = Train[['ip.len', 'tcp.flags.syn', 'tcp.flags.ack', 'tcp.flags.push',
                    'tcp.flags.fin', 'tcp.flags.reset', 'tcp.flags.ece', 'ip.proto','srcport', 'dstport']]
    if classification_type == "binary":
        y_train = Train['label']
    elif classification_type == "multiclass":
       y_train = Train['type'].replace(m_classes, range(len(m_classes)))
    elif classification_type == "4classes":
        y_train = Train['g_class'].replace(g_classes, range(len(g_classes)))
    return X_train, y_train

In [11]:
"""
Function to get test data features and labels
"""
def get_xtest_ytest(Test, classification_type, feat_num):

    if feat_num == 16:
        X_test = Test[['ip.len', 'tcp.flags.syn', 'tcp.flags.ack', 'tcp.flags.push',
                    'tcp.flags.fin', 'tcp.flags.reset', 'tcp.flags.ece', 'ip.proto', 'ip.ttl', 'tcp.window_size_value',
                    'tcp.hdr_len', 'udp.length','srcport', 'dstport']]
    else:
        X_test = Test[['ip.len', 'tcp.flags.syn', 'tcp.flags.ack', 'tcp.flags.push',
                    'tcp.flags.fin', 'tcp.flags.reset', 'tcp.flags.ece', 'ip.proto','srcport', 'dstport']]
    if classification_type == "binary":
        y_test = Test['label']
    elif classification_type == "multiclass":
        y_test = Test['type'].replace(m_classes, range(len(m_classes)))
    elif classification_type == "4classes":
        y_test = Test['g_class'].replace(g_classes, range(len(g_classes)))
    
    return X_test, y_test

In [12]:
"""
Function to perform a exhaustive search on the depth of the trees and 
the number of trees using different combinations of features
"""
def analyze_models(classes, model_type, depths, n_trees, X_train, y_train, X_test, y_test, max_leaf):
    if model_type == 'RF':
        # FOR EACH (depth, n_tree, feat)
        for depth in depths:
            for n_tree in n_trees:
                # get feature orders to use
                importance = get_feature_importance(depth, n_tree, max_leaf, X_train, y_train)
                #print("Feature importance of the model with (", depth, ", ", n_tree, ") is ", importance)
                m_feats = get_fewest_features(depth, n_tree, max_leaf, importance) 
                for feats in m_feats:
                    # Get the scores with the given (depth, n_tree, feat)
                    model, c_report, macro_f1, weight_f1, y_pred = get_scores(classes, depth, n_tree, feats, max_leaf, X_train, y_train, X_test, y_test)
                    print('Depth: ', depth, ' Number of Trees: ', n_tree, ' Features: ', feats, ' Macro Score: ', macro_f1, ' Weighted Score: ', weight_f1)

In [13]:
"""
Function to modify test data fields
"""
def prepare_test_data(test_data):
    test_data['tcp.flags.syn'] = test_data['tcp.flags.syn'].astype('Int64').fillna(0)
    test_data['tcp.flags.ack'] = test_data['tcp.flags.ack'].astype('Int64').fillna(0)
    test_data['tcp.flags.push'] = test_data['tcp.flags.push'].astype('Int64').fillna(0)
    test_data['tcp.flags.reset'] = test_data['tcp.flags.reset'].astype('Int64').fillna(0)
    test_data['tcp.flags.fin'] = test_data['tcp.flags.fin'].astype('Int64').fillna(0)
    test_data['tcp.flags.ece'] = test_data['tcp.flags.ece'].astype('Int64').fillna(0)
    test_data['ip.len'] = test_data['ip.len'].astype('Int64')
    test_data['ip.ttl'] = test_data['ip.ttl'].astype('Int64')
    test_data['ip.hdr_len'] = test_data['ip.hdr_len'].astype('Int64')
    test_data['tcp.window_size_value'] = test_data['tcp.window_size_value'].astype('Int64').fillna(0)
    test_data['tcp.hdr_len'] = test_data['tcp.hdr_len'].astype('Int64').fillna(0)
    test_data['udp.length'] = test_data['udp.length'].astype('Int64').fillna(0)
    test_data['srcport'] = test_data['srcport'].astype('Int64')
    test_data['dstport'] = test_data['dstport'].astype('Int64')

    return test_data

In [14]:
"""
Function to modify train data fields
"""
def prepare_train_data(train_data):
    train_data['tcp.flags.syn'] = train_data['tcp.flags.syn'].astype('Int64').fillna(0)
    train_data['tcp.flags.ack'] = train_data['tcp.flags.ack'].astype('Int64').fillna(0)
    train_data['tcp.flags.push'] = train_data['tcp.flags.push'].astype('Int64').fillna(0)
    train_data['tcp.flags.reset'] = train_data['tcp.flags.reset'].astype('Int64').fillna(0)
    train_data['tcp.flags.fin'] = train_data['tcp.flags.fin'].astype('Int64').fillna(0)
    train_data['tcp.flags.ece'] = train_data['tcp.flags.ece'].astype('Int64').fillna(0)
    train_data['ip.len'] = train_data['ip.len'].astype('Int64')
    train_data['ip.ttl'] = train_data['ip.ttl'].astype('Int64')
    train_data['ip.hdr_len'] = train_data['ip.hdr_len'].astype('Int64')
    train_data['tcp.window_size_value'] = train_data['tcp.window_size_value'].astype('Int64').fillna(0)
    train_data['tcp.hdr_len'] = train_data['tcp.hdr_len'].astype('Int64').fillna(0)
    train_data['udp.length'] = train_data['udp.length'].astype('Int64').fillna(0)
    train_data['srcport'] = train_data['srcport'].astype('Int64')
    train_data['dstport'] = train_data['dstport'].astype('Int64')
    train_data = train_data.drop(['eth.src', 'eth.dst', 'ID', 'Unnamed: 0', 'frame.time_relative', 'ip.src', 'ip.dst'], axis=1)
    
    return train_data

### Train Data

In [15]:
train_data = pd.read_csv("/home/nds-admin/experiments/MetaCom/dataset/ToN_IoT/data/train.csv")

In [16]:
train_data = prepare_train_data(train_data)

In [21]:
train_data.loc[:,'tcp.hdr_len'] /= 4
train_data['tcp.hdr_len'] = train_data['tcp.hdr_len'].astype('Int64')

### Test Data

In [25]:
test_data = pd.read_csv("/home/nds-admin/experiments/MetaCom/dataset/ToN_IoT/data/test.csv")

In [27]:
test_data = prepare_test_data(test_data)

In [29]:
test_data.loc[:,'tcp.hdr_len'] /= 4
test_data['tcp.hdr_len'] = test_data['tcp.hdr_len'].astype('Int64')

### Data Statistics

In [32]:
len(test_data)/ (len(test_data) + len(train_data))

0.25991122281316636

In [33]:
len(test_data[test_data['type'] == 'normal'])/len(test_data)

0.36423612276395817

# Modify test and train data for 4 Classes:
### benign, backdoor, dos, others

In [34]:
train_data_4classes = prepare_new_classes(train_data)

In [35]:
train_data_4classes['g_class'].value_counts()

others      49772029
normal      24741942
dos          8860573
backdoor      369013
Name: g_class, dtype: int64

In [36]:
test_data_4classes = prepare_new_classes(test_data)

In [37]:
test_data_4classes['g_class'].value_counts()

others      14572422
normal      10712125
dos          3958692
backdoor      166598
Name: g_class, dtype: int64

In [38]:
# Generate train and test data for 4 classes
X_train, y_train = get_xtrain_ytrain(train_data_4classes, "4classes", 16)
X_test, y_test = get_xtest_ytest(test_data_4classes, "4classes", 16)

### Analyze the models with different depths and number of trees and with different combinations of features

In [ ]:
# This runs the exhaustive grid search on different depths and number of trees and
# We pick the best model according to desired F1 score
analyze_models(g_classes, "RF", [6,7,8,9,10,11,12,13,14], [2,3,4,5,6], X_train, y_train, X_test, y_test, 524)

### Performance of the best models we chose according to score and size

##### Depth: 5 Tree: 3 Feats: 7

In [46]:
feats_4classes =['srcport', 'dstport', 'tcp.window_size_value', 'ip.ttl', 'tcp.hdr_len', 'tcp.flags.ack', 'tcp.flags.syn']
model, class_report, macro_score, weighted_score, y_pred =  get_scores(g_classes, 5, 3, feats_4classes, 524, X_train, y_train, X_test, y_test)
class_report

{'normal': {'precision': 0.9938669042300264,
  'recall': 0.9715764145769397,
  'f1-score': 0.9825952589308212,
  'support': 10712125},
 'backdoor': {'precision': 0.9817152966265995,
  'recall': 0.9726287230338899,
  'f1-score': 0.9771508861644967,
  'support': 166598},
 'dos': {'precision': 0.9720560613764483,
  'recall': 0.9991871052357698,
  'f1-score': 0.9854348753561809,
  'support': 3958692},
 'others': {'precision': 0.9791987404552248,
  'recall': 0.9880216891879744,
  'f1-score': 0.9835904294373163,
  'support': 14572422},
 'accuracy': 0.9834474431123165,
 'macro avg': {'precision': 0.9817092506720748,
  'recall': 0.9828534830086435,
  'f1-score': 0.9821928624722038,
  'support': 29409837},
 'weighted avg': {'precision': 0.9835942354242538,
  'recall': 0.9834474431123165,
  'f1-score': 0.9834397447331267,
  'support': 29409837}}

In [39]:
#save_model(model, "model_d5t3f7.sav")

In [47]:
# Confusion matrix of 4 classes: benign, backdoor, dos, others
conf_matrix = confusion_matrix(test_data['g_class'].replace(g_classes, range(len(g_classes))), y_pred)

In [48]:
conf_matrix

array([[10407648,     2936,        0,   301541],
       [    3163,   162038,        0,     1397],
       [     300,        0,  3955474,     2918],
       [   60762,       82,   113709, 14397869]])

In [49]:
# Classification results
classification_results = test_data
classification_results["class_pred"] = y_pred
classification_results = classification_results[["g_class", "class_pred"]]
classification_results.rename(columns={'g_class':'class'}, inplace=True)
classification_results["class"] = classification_results["class"].replace(g_classes, range(len(g_classes)))

In [50]:
# Performance Values calculated in terms of classifying benign and malicious traffic
T_P = classification_results[((classification_results['class_pred'] != 0) &  (classification_results['class'] != 0))]
F_P = classification_results[((classification_results['class_pred'] != 0) &  (classification_results['class'] == 0))]

T_N = classification_results[((classification_results['class_pred'] == 0) &  (classification_results['class'] == 0))]
F_N = classification_results[((classification_results['class_pred'] == 0) &  (classification_results['class'] != 0))]

print("TP: ", len(T_P), " TPR => ", len(T_P)/(len(T_P) + len(F_N)))
print("FP: ", len(F_P), " FPR => ", len(F_P)/(len(F_P) + len(T_N)))
print("TN: ", len(T_N), " TNR => ", len(T_N)/(len(T_N) + len(F_P)))
print("FN: ", len(F_N), " FNR => ", len(F_N)/(len(F_N) + len(T_P)))

TP:  18633487  TPR =>  0.9965650877497739
FP:  304477  FPR =>  0.028423585423060317
TN:  10407648  TNR =>  0.9715764145769397
FN:  64225  FNR =>  0.0034349122502261237


##### Depth: 7 Tree: 3 Feats: 7

In [40]:
feats_4classes = ['srcport', 'tcp.window_size_value', 'dstport', 'ip.ttl', 'tcp.hdr_len', 'tcp.flags.ack', 'tcp.flags.syn']
model, class_report, macro_score, weighted_score, y_pred =  get_scores(g_classes, 7, 3, feats_4classes, 524, X_train, y_train, X_test, y_test)
class_report

{'normal': {'precision': 0.9717685961512563,
  'recall': 0.9888849317945786,
  'f1-score': 0.9802520519083741,
  'support': 10712125},
 'backdoor': {'precision': 0.9983441969664205,
  'recall': 0.9699216077023733,
  'f1-score': 0.9839276852395928,
  'support': 166598},
 'dos': {'precision': 1.0,
  'recall': 0.9991871052357698,
  'f1-score': 0.9995933873512383,
  'support': 3958692},
 'others': {'precision': 0.9913655116864549,
  'recall': 0.9790712209679352,
  'f1-score': 0.9851800119913855,
  'support': 14572422},
 'accuracy': 0.9853015846364602,
 'macro avg': {'precision': 0.9903695762010329,
  'recall': 0.9842662164251643,
  'f1-score': 0.9872382841226476,
  'support': 29409837},
 'weighted avg': {'precision': 0.985429379024234,
  'recall': 0.9853015846364602,
  'f1-score': 0.9853180798832144,
  'support': 29409837}}

In [41]:
#save_model(model, "model_d7t3f7.sav")

In [42]:
# Confusion matrix of 4 classes: benign, backdoor, dos, others
conf_matrix = confusion_matrix(test_data['g_class'].replace(g_classes, range(len(g_classes))), y_pred)

In [43]:
conf_matrix

array([[10593059,      268,        0,   118798],
       [    2462,   161587,        0,     2549],
       [     300,        0,  3955474,     2918],
       [  304983,        0,        0, 14267439]])

In [44]:
# Classification results
classification_results = test_data
classification_results["class_pred"] = y_pred
classification_results = classification_results[["g_class", "class_pred"]]
classification_results.rename(columns={'g_class':'class'}, inplace=True)
classification_results["class"] = classification_results["class"].replace(g_classes, range(len(g_classes)))


In [45]:
# Performance Values calculated in terms of classifying benign and malicious traffic
T_P = classification_results[((classification_results['class_pred'] != 0) &  (classification_results['class'] != 0))]
F_P = classification_results[((classification_results['class_pred'] != 0) &  (classification_results['class'] == 0))]

T_N = classification_results[((classification_results['class_pred'] == 0) &  (classification_results['class'] == 0))]
F_N = classification_results[((classification_results['class_pred'] == 0) &  (classification_results['class'] != 0))]

print("TP: ", len(T_P), " TPR => ", len(T_P)/(len(T_P) + len(F_N)))
print("FP: ", len(F_P), " FPR => ", len(F_P)/(len(F_P) + len(T_N)))
print("TN: ", len(T_N), " TNR => ", len(T_N)/(len(T_N) + len(F_P)))
print("FN: ", len(F_N), " FNR => ", len(F_N)/(len(F_N) + len(T_P)))

TP:  18389967  TPR =>  0.9835410343254832
FP:  119066  FPR =>  0.011115068205421427
TN:  10593059  TNR =>  0.9888849317945786
FN:  307745  FNR =>  0.016458965674516754


##### Depth: 9 Tree: 3 Feats: 10

In [51]:
feats_4classes = ['dstport', 'tcp.window_size_value', 'srcport', 'ip.ttl', 'tcp.hdr_len', 'tcp.flags.ack', 'tcp.flags.syn', 'tcp.flags.push', 'ip.len', 'udp.length']
model, class_report, macro_score, weighted_score, y_pred =  get_scores(g_classes, 9, 3, feats_4classes, 524, X_train, y_train, X_test, y_test)
class_report

{'normal': {'precision': 0.9980930217679588,
  'recall': 0.9728429233228701,
  'f1-score': 0.9853062302338302,
  'support': 10712125},
 'backdoor': {'precision': 0.996426701167483,
  'recall': 0.9708099737091682,
  'f1-score': 0.9834515510180504,
  'support': 166598},
 'dos': {'precision': 0.9999989887442176,
  'recall': 0.9991871052357698,
  'f1-score': 0.9995928821341974,
  'support': 3958692},
 'others': {'precision': 0.9800029196848302,
  'recall': 0.9987319197865667,
  'f1-score': 0.9892787834419885,
  'support': 14572422},
 'accuracy': 0.9892053124945915,
 'macro avg': {'precision': 0.9936304078411224,
  'recall': 0.9853929805135937,
  'f1-score': 0.9894073617070167,
  'support': 29409837},
 'weighted avg': {'precision': 0.9893765820598215,
  'recall': 0.9892053124945915,
  'f1-score': 0.9891871490741323,
  'support': 29409837}}

In [41]:
#save_model(model, "model_d9t3f10.sav")

In [52]:
# Confusion matrix of 4 classes: benign, backdoor, dos, others
conf_matrix = confusion_matrix(test_data['g_class'].replace(g_classes, range(len(g_classes))), y_pred)

In [53]:
conf_matrix

array([[10421215,      580,        0,   290330],
       [    1418,   161735,        4,     3441],
       [      14,        0,  3955474,     3204],
       [   18479,        0,        0, 14553943]])

In [54]:
# Classification results
classification_results = test_data
classification_results["class_pred"] = y_pred
classification_results = classification_results[["g_class", "class_pred"]]
classification_results.rename(columns={'g_class':'class'}, inplace=True)
classification_results["class"] = classification_results["class"].replace(g_classes, range(len(g_classes)))

In [55]:
# Performance Values calculated in terms of classifying benign and malicious traffic
T_P = classification_results[((classification_results['class_pred'] != 0) &  (classification_results['class'] != 0))]
F_P = classification_results[((classification_results['class_pred'] != 0) &  (classification_results['class'] == 0))]

T_N = classification_results[((classification_results['class_pred'] == 0) &  (classification_results['class'] == 0))]
F_N = classification_results[((classification_results['class_pred'] == 0) &  (classification_results['class'] != 0))]

print("TP: ", len(T_P), " TPR => ", len(T_P)/(len(T_P) + len(F_N)))
print("FP: ", len(F_P), " FPR => ", len(F_P)/(len(F_P) + len(T_N)))
print("TN: ", len(T_N), " TNR => ", len(T_N)/(len(T_N) + len(F_P)))
print("FN: ", len(F_N), " FNR => ", len(F_N)/(len(F_N) + len(T_P)))

TP:  18677801  TPR =>  0.9989351103493305
FP:  290910  FPR =>  0.02715707667712989
TN:  10421215  TNR =>  0.9728429233228701
FN:  19911  FNR =>  0.0010648896506695578


##### Depth: 10 Tree: 3 Feats: 9

In [56]:
feats_4classes = ['dstport', 'tcp.window_size_value', 'srcport', 'ip.ttl', 'tcp.hdr_len', 'tcp.flags.ack', 'tcp.flags.syn', 'tcp.flags.push', 'ip.len']
model, class_report, macro_score, weighted_score, y_pred =  get_scores(g_classes, 10, 3, feats_4classes, 524, X_train, y_train, X_test, y_test)
class_report

{'normal': {'precision': 0.9986985759757463,
  'recall': 0.9926081893181792,
  'f1-score': 0.9956440689615264,
  'support': 10712125},
 'backdoor': {'precision': 0.9995860952722195,
  'recall': 0.9712361492935089,
  'f1-score': 0.9852072176843617,
  'support': 166598},
 'dos': {'precision': 0.9720529559156158,
  'recall': 0.9991871052357698,
  'f1-score': 0.9854332795873585,
  'support': 3958692},
 'others': {'precision': 0.9941175211700493,
  'recall': 0.991357853896902,
  'f1-score': 0.9927357696644468,
  'support': 14572422},
 'accuracy': 0.9927531390262381,
 'macro avg': {'precision': 0.9911137870834077,
  'recall': 0.9885973244360899,
  'f1-score': 0.9897550839744234,
  'support': 29409837},
 'weighted avg': {'precision': 0.992847098190913,
  'recall': 0.9927531390262381,
  'f1-score': 0.992769483379239,
  'support': 29409837}}

In [43]:
#save_model(model, "model_d10t3f9.sav")

In [57]:
# Confusion matrix of 4 classes: benign, backdoor, dos, others
conf_matrix = confusion_matrix(test_data['g_class'].replace(g_classes, range(len(g_classes))), y_pred)

In [58]:
conf_matrix

array([[10632943,       64,        3,    79115],
       [    1622,   161806,       10,     3160],
       [       9,        0,  3955474,     3209],
       [   12225,        3,   113709, 14446485]])

In [59]:
#### Classification results
classification_results = test_data
classification_results["class_pred"] = y_pred
classification_results = classification_results[["g_class", "class_pred"]]
classification_results.rename(columns={'g_class':'class'}, inplace=True)
classification_results["class"] = classification_results["class"].replace(g_classes, range(len(g_classes)))

In [60]:
# Performance Values calculated in terms of classifying benign and malicious traffic
T_P = classification_results[((classification_results['class_pred'] != 0) &  (classification_results['class'] != 0))]
F_P = classification_results[((classification_results['class_pred'] != 0) &  (classification_results['class'] == 0))]

T_N = classification_results[((classification_results['class_pred'] == 0) &  (classification_results['class'] == 0))]
F_N = classification_results[((classification_results['class_pred'] == 0) &  (classification_results['class'] != 0))]

print("TP: ", len(T_P), " TPR => ", len(T_P)/(len(T_P) + len(F_N)))
print("FP: ", len(F_P), " FPR => ", len(F_P)/(len(F_P) + len(T_N)))
print("TN: ", len(T_N), " TNR => ", len(T_N)/(len(T_N) + len(F_P)))
print("FN: ", len(F_N), " FNR => ", len(F_N)/(len(F_N) + len(T_P)))

TP:  18683856  TPR =>  0.999258946763112
FP:  79182  FPR =>  0.007391810681820834
TN:  10632943  TNR =>  0.9926081893181792
FN:  13856  FNR =>  0.0007410532368880213
